In [1]:
import os 
import pandas as pd

In [2]:
path = os.getcwd()
path = path + '\dataset'
df = pd.read_csv(path + '\Fake news classifier train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df.shape

(20800, 5)

In [5]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [6]:
df = df.dropna()

In [7]:
df.shape

(18285, 5)

In [8]:
## get the independent feautures
X = df.drop('label', axis=1)
y = df['label']

In [9]:
## Checkking whether dataset is balanced or not
y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [10]:
import tensorflow as tf
print(tf.__version__)

2.11.0


In [11]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense## for creating nuerons
from tensorflow.keras.layers import Bidirectional

In [12]:
voc_size = 5000

In [13]:
messages=X.copy()
messages.reset_index(inplace=True) ##required because we removed empty rows

In [14]:
messages.head()

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [15]:
import nltk
import re

In [16]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lmbmo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:
from nltk.corpus import stopwords

In [18]:
## Data preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(len(messages)):
    review = re.sub('[^a-zA-Z]',' ', messages['title'][i])
    review = review.lower().split()
    
    review = [ps.stem(word) for word in review if word not in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [19]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [20]:
# one hot encodding
onehot_repr = [one_hot(words, voc_size) for words in corpus]
onehot_repr

[[3567, 4459, 1930, 416, 4297, 4589, 2544, 1405, 1862, 3226],
 [3379, 2641, 556, 166, 2865, 3492, 1160],
 [1, 4597, 93, 1078],
 [621, 3352, 2060, 4758, 4519, 2839],
 [263, 2865, 610, 3497, 4968, 2515, 2865, 1261, 586, 1229],
 [217,
  1589,
  2185,
  4391,
  1832,
  2342,
  3274,
  2113,
  4613,
  2442,
  1674,
  1292,
  4200,
  4774,
  1160],
 [208, 374, 3532, 4709, 4334, 2214, 2076, 3994, 4791, 1639, 4468],
 [4427, 672, 1701, 234, 123, 2218, 2342, 1872, 4791, 1639, 4468],
 [4360, 1239, 3405, 1861, 4511, 2331, 1307, 541, 2342, 2351],
 [379, 4892, 639, 596, 7, 573, 1013, 1160],
 [373, 4701, 3973, 3604, 3814, 2266, 3488, 1507, 4899, 3643, 4352],
 [4758, 37, 4297, 2331, 2342, 123],
 [4180, 2666, 3442, 1802, 1623, 4693, 2240, 516, 2532],
 [4257, 4611, 319, 4435, 2136, 2944, 3387, 4791, 1639, 4468],
 [4439, 133, 3097, 1840, 1589, 4791, 1639, 4468],
 [38, 2754, 150, 2734, 879, 2997, 315, 2588, 436, 4579],
 [1393, 182, 2641],
 [1843, 213, 1313, 2612, 2342, 1871, 3565, 1160],
 [3070, 1271, 556

In [21]:
print(corpus[0])
print(onehot_repr[0])

hous dem aid even see comey letter jason chaffetz tweet
[3567, 4459, 1930, 416, 4297, 4589, 2544, 1405, 1862, 3226]


In [22]:
## Embedding representation
sent_length = 20
embedded_docs =pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)

In [23]:
embedded_docs

array([[   0,    0,    0, ..., 1405, 1862, 3226],
       [   0,    0,    0, ..., 2865, 3492, 1160],
       [   0,    0,    0, ..., 4597,   93, 1078],
       ...,
       [   0,    0,    0, ..., 4791, 1639, 4468],
       [   0,    0,    0, ..., 3814, 3564, 2691],
       [   0,    0,    0, ..., 2561, 1331, 1770]])

In [24]:
## creating model
embedded_vector_features=40#number of features
model=Sequential()
model.add(Embedding(voc_size,embedded_vector_features,input_length=sent_length))
model.add(Bidirectional(LSTM(100)))#number of nuerons
model.add(Dense(1,activation='sigmoid'))# as it is binary classifier
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 bidirectional (Bidirectiona  (None, 200)              112800    
 l)                                                              
                                                                 
 dense (Dense)               (None, 1)                 201       
                                                                 
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [25]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [26]:
## train_test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X_final, y_final, test_size=0.3, random_state=33)

In [29]:
##model training
model.fit(X_train, y_train, validation_data=(X_test, y_test),epochs=40, batch_size=32)

Epoch 1/40
400/400 [==============================] - 9s 23ms/step - loss: 9.1757e-06 - accuracy: 1.0000 - val_loss: 0.9096 - val_accuracy: 0.9092
Epoch 2/40
400/400 [==============================] - 9s 23ms/step - loss: 7.1323e-06 - accuracy: 1.0000 - val_loss: 0.9267 - val_accuracy: 0.9092
Epoch 3/40
400/400 [==============================] - 9s 23ms/step - loss: 5.7904e-06 - accuracy: 1.0000 - val_loss: 0.9425 - val_accuracy: 0.9089
Epoch 4/40
400/400 [==============================] - 9s 23ms/step - loss: 4.5522e-06 - accuracy: 1.0000 - val_loss: 0.9595 - val_accuracy: 0.9089
Epoch 5/40
400/400 [==============================] - 9s 23ms/step - loss: 3.5952e-06 - accuracy: 1.0000 - val_loss: 0.9759 - val_accuracy: 0.9090
Epoch 6/40
400/400 [==============================] - 9s 24ms/step - loss: 2.8645e-06 - accuracy: 1.0000 - val_loss: 0.9917 - val_accuracy: 0.9090
Epoch 7/40
400/400 [==============================] - 10s 24ms/step - loss: 2.2592e-06 - accuracy: 1.0000 - val_loss: 

In [30]:
y_pred=model.predict(X_test)

172/172 [==============================] - 2s 8ms/step


In [31]:
y_pred=np.where(y_pred>=0.5,1,0)

In [32]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
conf_mat = confusion_matrix(y_test,y_pred)
acc = accuracy_score(y_test,y_pred)
class_rep = classification_report(y_test,y_pred)

print(conf_mat)
print(f'Accuracy is {acc}')
print(class_rep)

[[2866  290]
 [ 211 2119]]
Accuracy is 0.9086766314254466
              precision    recall  f1-score   support

           0       0.93      0.91      0.92      3156
           1       0.88      0.91      0.89      2330

    accuracy                           0.91      5486
   macro avg       0.91      0.91      0.91      5486
weighted avg       0.91      0.91      0.91      5486

